Data Privacy Final Project 

In [2]:
# https://www.kaggle.com/code/marosz/math-final-grade-correlation-and-comparison#Comparisons:-the-students-whose-final-grade-is-75%-or-better-vs.-the-students-whose-final-grade-is-50%-or-worse
# Load the data and libraries
import pandas as pd
import numpy as np
import random
from scipy import stats
import matplotlib.pyplot as plt

def laplace_mech(v, sensitivity, epsilon):
    return v + np.random.laplace(loc=0, scale=sensitivity / epsilon)

def laplace_mech_vec(vec, sensitivity, epsilon):
    return [v + np.random.laplace(loc=0, scale=sensitivity / epsilon) for v in vec]

def gaussian_mech(v, sensitivity, epsilon, delta):
    return v + np.random.normal(loc=0, scale=sensitivity * np.sqrt(2*np.log(1.25/delta)) / epsilon)

def gaussian_mech_vec(vec, sensitivity, epsilon, delta):
    return [v + np.random.normal(loc=0, scale=sensitivity * np.sqrt(2*np.log(1.25/delta)) / epsilon)
            for v in vec]

def pct_error(orig, priv):
    return np.abs(orig - priv)/orig * 100.0

# preserves epsilon-differential privacy
def above_threshold(query_results, T, epsilon):
    T_hat = T + np.random.laplace(loc=0, scale = 2/epsilon)
    
    for idx, q in enumerate(query_results):
        nu_i = np.random.laplace(loc=0, scale = 4/epsilon)
        if q + nu_i >= T_hat:
            return idx
    return None

math = pd.read_csv("student_math_clean.csv")
math.head(5)

FileNotFoundError: [Errno 2] No such file or directory: 'student_math_clean.csv'

In [10]:
## Cache the sorted ages, because we will use them a lot.
absences_lower = 0
absences_upper = 20
sorted_absences = math['absences'].clip(lower=absences_lower, upper=absences_upper).sort_values()

#what is the upper bound of the data - b

# we wrote a function that found b - to know where to clip 
# privataize that clipped idea data under that 
# by apply gauss and laplace

def min_absences():
    clipped_absences = math['absences'].clip(lower=0, upper=100)
    return clipped_absences.min()

def max_absences():
    clipped_absences = math['absences'].clip(lower=0, upper=100)
    return clipped_absences.max()

def ls_min():
    return max(sorted_absences.iloc[0] - absences_lower, sorted_absences.iloc[1] - sorted_absences.iloc[0])

print('Actual minimum absences:', min_absences())
print('Local sensitivity of the minimum:', ls_min())
print('max', max_absences())


Actual minimum absences: 0
Local sensitivity of the minimum: 0
max 75


In [15]:
bs = list(range(0, 75, 5)) #(min, max, step)
df = math['absences']

#find upper cliping param for mean_abscences 
def upper(b_lower, epsilon):
        query_results = [math.clip(lower=b_lower, upper=b).sum() - \
                         math.clip(lower=b_lower, upper=b+1).sum() for b in bs]
        idx = above_threshold(query_results, 0, epsilon)
        
        if idx == None:
            return bs[-1]
        else:
            return bs[idx]

In [5]:
# finds upper and lower, and adds laplace differenial noise
def mean_abscence_lap(epsilon):
    b_lower = 0
    b_upper = upper(b_lower, epsilon / 3) #finds upper clipping param
    
    noisy_sum = laplace_mech(df.clip(lower=b_lower, upper=b_upper).sum(), np.abs(b_upper - b_lower), epsilon / 3)
    noisy_count = laplace_mech(len(df), 1, epsilon / 3)
    
    return noisy_sum / noisy_count

    
results = []

# Iter over epsilons and store results
for epsilon in [0.001, 0.01, 0.1, 0.5, 1, 10]:
    result = mean_abscence_lap(epsilon, math)
    results.append(result)
    print(f"epsilon: {epsilon}, mean abscence lap: {result}")

#from exersize 10-2
def cdf(data):
    n = len(data)
    x = np.sort(data)
    y = np.arange(1, n + 1) / n
    return x, y

# Plot the CDF for the results
for i, result in enumerate(results):
    x, y = cdf(result)
    plt.plot(x, y, label=f'Epsilon = {epsilon_values[i]}')

plt.xlabel('Mean Absence Laplace')
plt.ylabel('Cumulative Probability')
plt.title('Cumulative Distribution Function (CDF) of Mean Absence Laplace')
plt.legend()
plt.show()

NameError: name 'math' is not defined

In [20]:
def mean_abscence_gaussian(epsilon, math):
    b_lower = 0
    b_upper = upper(b_lower, epsilon / 3)  # finds upper clipping param

    noisy_sum = gaussian_mech(df.clip(lower=b_lower, upper=b_upper).sum(), np.abs(b_upper - b_lower), epsilon / 3, 0.0001)
    noisy_count = gaussian_mech(len(df), 1, epsilon / 3, 0.0001)

    return noisy_sum / noisy_count

# store results
results_gaussian = []

for epsilon in [0.001, 0.01, 0.1, 0.5, 1, 10]:
    result_gaussian = mean_abscence_gaussian(epsilon, math)
    results_gaussian.append(result_gaussian)
    print(f"epsilon: {epsilon}, mean abscence gaussian: {result_gaussian}")

# CDF fnc
def cdf(data):
    n = len(data)
    x = np.sort(data)
    y = np.arange(1, n + 1) / n
    return x, y

# Plot the CDF for the results
for i, result_gaussian in enumerate(results_gaussian):
    x, y = cdf(result_gaussian)
    plt.plot(x, y, label = f'Epsilon = {epsilon_values[i]}')

plt.xlabel('Mean Absence Gaussian')
plt.ylabel('Cumulative Probability')
plt.title('Cumulative Distribution Function (CDF) of Mean Absence Gaussian')
plt.legend()
plt.show()

epsilon: 0.001, mean abscence gaus: 1.6269915145814782
epsilon: 0.01, mean abscence gaus: 26.315337955534048
epsilon: 0.1, mean abscence gaus: 1.5312429172358721
epsilon: 0.5, mean abscence gaus: 8.100927410793556
epsilon: 1, mean abscence gaus: 4.714205591407119
epsilon: 10, mean abscence gaus: 5.346766158197703


In [6]:
# Function to calculate mean_abscence_lap for a given epsilon
def mean_abscence_lap(epsilon, math):
    b_lower = 0
    b_upper = upper(b_lower, epsilon / 3)  # finds upper clipping param

    noisy_sum = laplace_mech(df.clip(lower=b_lower, upper=b_upper).sum(), np.abs(b_upper - b_lower), epsilon / 3)
    noisy_count = laplace_mech(len(df), 1, epsilon / 3)

    return noisy_sum / noisy_count

# Function to calculate mean_abscence_gaussian for a given epsilon
def mean_abscence_gaussian(epsilon, math):
    b_lower = 0
    b_upper = upper(b_lower, epsilon / 3)  # finds upper clipping param

    noisy_sum = gaussian_mech(df.clip(lower=b_lower, upper=b_upper).sum(), np.abs(b_upper - b_lower), epsilon / 3, 0.0001)
    noisy_count = gaussian_mech(len(df), 1, epsilon / 3, 0.0001)

    return noisy_sum / noisy_count

# Create a list to store Laplace results
results_laplace = []

# Create a list to store Gaussian results
results_gaussian = []

# Define your DataFrame 'df' here

# Iterate over epsilon values and store Laplace and Gaussian results
epsilon_values = [0.001, 0.01, 0.1, 0.5, 1, 10]
for epsilon in epsilon_values:
    result_laplace = mean_abscence_lap(epsilon, math)
    result_gaussian = mean_abscence_gaussian(epsilon, math)
    results_laplace.append(result_laplace)
    results_gaussian.append(result_gaussian)
    print(f"epsilon: {epsilon}, mean abscence lap: {result_laplace}, mean abscence gaussian: {result_gaussian}")

# CDF function
def cdf(data):
    n = len(data)
    x = np.sort(data)
    y = np.arange(1, n + 1) / n
    return x, y

# Plot the CDF for Laplace results
for i, result_laplace in enumerate(results_laplace):
    x, y = cdf(result_laplace)
    plt.plot(x, y, label=f'Laplace, Epsilon = {epsilon_values[i]}', linestyle='--')

# Plot the CDF for Gaussian results
for i, result_gaussian in enumerate(results_gaussian):
    x, y = cdf(result_gaussian)
    plt.plot(x, y, label=f'Gaussian, Epsilon = {epsilon_values[i]}')

plt.xlabel('Mean Absence')
plt.ylabel('Cumulative Probability')
plt.title('Comparison of Laplace and Gaussian Mechanisms')
plt.legend()
plt.show()

NameError: name 'df' is not defined

In [6]:
# not here!
# preserves epsilon-differential privacy
def above_threshold(query_results, T, epsilon):
    T_hat = T + np.random.laplace(loc=0, scale = 2/epsilon)
    
    for idx, q in enumerate(query_results):
        nu_i = np.random.laplace(loc=0, scale = 4/epsilon)
        if q + nu_i >= T_hat:
            return idx
    return None
def above_15(query_results, epsilon):
    ### BEGIN SOLUTION
    T = 15
    idx = above_threshold(query_results, T, epsilon/2)
    val = laplace_mech(query_results[idx], 1, epsilon/2)
    return val
    ### END SOLUTION

queries = math['absences'].value_counts()
print(f"above_15 #1: {above_15(queries, 100)}")
print(f"above_15 #2: {above_15(queries, 1)}")
print(f"above_15 #3: {above_15(queries, .01)}")

above_15 #1: 114.99887956057044
above_15 #2: 111.90112217102819
above_15 #3: 30.240166167322588


In [12]:
def bounded_all_above_10000(query_results, c, epsilon):
    ### BEGIN SOLUTION
    answers = []
    pos = 0
    T = 15
    epsilon_i = epsilon / (c*2)
    
    while len(answers) < c:
        answer_idx = above_threshold(query_results[pos:], T, epsilon_i)
        if answer_idx == None:
            break
        pos = pos + answer_idx
        answers.append(laplace_mech(query_results[pos], 1, epsilon_i))
        pos = pos + 1
        
    return answers
    ### END SOLUTION

queries = list(math['absences'].value_counts())
print(f"bounded_all_above_10000 #1: {bounded_all_above_15(queries, 3, 100)}")
print(f"bounded_all_above_10000 #2: {bounded_all_above_15(queries, 3, 1)}")
print(f"bounded_all_above_10000 #3: {bounded_all_above_10000(queries, 3, .01)}")

NameError: name 'bounded_all_above_15' is not defined